In [1]:
%load_ext autoreload
%autoreload 2
%gui qt
%matplotlib qt

import sys

import numpy as np
import matplotlib.pyplot as plt
import torch
import oyDL

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Create train, test, and val datasets from directory

In [148]:
from oyDL import trainset
data_dir_train = ['/bigstore/Microscopy Core/Jen/20220104_DICCytotoxDrugsTL/predeathzoom/necro/*',
                  '/bigstore/Microscopy Core/Jen/20220104_DICCytotoxDrugsTL/predeathzoom/apo/*',
                  '/bigstore/Microscopy Core/Jen/20220104_DICCytotoxDrugsTL/predeathzoom/pyro/*',
                ]

dataloaders, dataloader_full, dataset_sizes, class_names = trainset(data_dir_train)

### Look at some images

In [3]:
import torchvision

# Get a batch of training data
inputs, classes, labels = next(iter(dataloaders['train']))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs,normalize=True)

oyDL.imshow(out, title=[class_names[x] for x in classes])

### Load pretrained resnet model from web

In [149]:
from oyDL import ResNet
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

Res = ResNet(network='resnet50', pretrained=True,inchans=1, device=device)

#should all the layers aside from fc be frozen?
freezeflag = 0
if freezeflag:
    for param in Res.model.parameters():
        param.requires_grad = False

# Add fc layer with len(class_names) features. This is the only layer that will be trained if freezeflag==1
Res.model.fc = nn.Linear(Res.model.fc.in_features, len(class_names))



In [150]:
model_ft = Res.train_model(dataloaders, num_epochs=25)

Epoch 0/24
----------
train Loss: 0.8587 Acc: 0.6118
val Loss: 0.8631 Acc: 0.6585

Epoch 1/24
----------
train Loss: 0.5740 Acc: 0.7636
val Loss: 0.5885 Acc: 0.7817

Epoch 2/24
----------
train Loss: 0.4872 Acc: 0.8051
val Loss: 0.7794 Acc: 0.6796

Epoch 3/24
----------
train Loss: 0.4106 Acc: 0.8421
val Loss: 0.9533 Acc: 0.6127

Epoch 4/24
----------
train Loss: 0.3735 Acc: 0.8565
val Loss: 0.6626 Acc: 0.7500

Epoch 5/24
----------
train Loss: 0.2784 Acc: 0.8973
val Loss: 0.5020 Acc: 0.8063

Epoch 6/24
----------
train Loss: 0.2006 Acc: 0.9313
val Loss: 0.8412 Acc: 0.6620

Epoch 7/24
----------
train Loss: 0.1758 Acc: 0.9381
val Loss: 0.8555 Acc: 0.6620

Epoch 8/24
----------
train Loss: 0.1821 Acc: 0.9305
val Loss: 0.7796 Acc: 0.7359

Epoch 9/24
----------
train Loss: 0.1713 Acc: 0.9449
val Loss: 0.5787 Acc: 0.7746

Epoch 10/24
----------
train Loss: 0.1638 Acc: 0.9479
val Loss: 0.6521 Acc: 0.7324

Epoch 11/24
----------
train Loss: 0.1670 Acc: 0.9396
val Loss: 0.5768 Acc: 0.7958

Ep

## You can save your model

In [137]:
fname = '/bigstore/Microscopy Core/Jen/20220104_DICCytotoxDrugsTL/trained_resnet_celldeath.res'
Res.save(filename=fname)

saved model


## And load your model

In [146]:
from oyDL import ResNet
fname = '/bigstore/Microscopy Core/Jen/20220104_DICCytotoxDrugsTL/trained_resnet_celldeath.res'
Res2 = ResNet(device=device, filename=fname)

loaded model


## Another way to load

In [ ]:
from oyDL import ResNet
fname = '/bigstore/Microscopy Core/Jen/20220104_DICCytotoxDrugsTL/trained_resnet_celldeath.res'
Res2 = ResNet.load(fname)

# Do some plotting

#### logit scores in 2D

In [151]:
outcp ,labels= Res.get_logits_and_labels(dataloaders['train'])

fig = plt.figure()
ax=fig.add_subplot(111)

ax.scatter(outcp[:,0],outcp[:,1],c=labels, alpha=0.2)
plt.xlabel(class_names[0])
plt.ylabel(class_names[1])

Text(0, 0.5, 'necro')

#### logit scores in 3D

In [122]:
from mpl_toolkits.mplot3d import Axes3D
outcp ,labelsssss= Res.get_logits_and_labels(dataloaders['test'])

fig = plt.figure()
ax=fig.add_subplot(111,projection='3d')
ax.scatter(outcp[:,0],outcp[:,1], outcp[:,2],c=labelsssss, alpha=0.2)
plt.xlabel(class_names[0])
plt.ylabel(class_names[1])


Text(0.5, 0.5, 'necro')

# Check accuracy

In [152]:
predicts, labels = Res.get_predicts_and_labels(dataloaders['test'])
print(class_names)
print(np.sum(np.column_stack((predicts==0, predicts==1,predicts==2)), axis=0)/len(predicts))
print(np.sum(np.column_stack((labels==0, labels==1,labels==2)), axis=0)/len(labels))


['apo', 'necro', 'pyro']
[0.31690141 0.25704225 0.42605634]
[0.31338028 0.30985915 0.37676056]


# confusion matrix

In [153]:
Res.ConfusionMatrix(dataloaders['val'])

In [154]:
plt.close('all')

# Classification report

In [36]:
Res.classification_report(dataloaders['test'])

              precision    recall  f1-score   support

         apo       0.90      0.77      0.83       105
       necro       0.76      0.83      0.79       109
        pyro       0.76      0.80      0.78        70

    accuracy                           0.80       284
   macro avg       0.81      0.80      0.80       284
weighted avg       0.81      0.80      0.80       284



### UMAP embedding of final logit layer

In [154]:
activations, labels = Res.getActivationsByType(dataloader_full)

a = np.squeeze(np.array(activations['avgpool'].cpu()))

import umap

umapfit = umap.UMAP(n_neighbors=20,
                      min_dist=0.05,
                      metric='euclidean').fit(a)

embedding = umapfit.embedding_

fig, ax = plt.subplots()

lbls = np.unique(labels)

scatter = ax.scatter(embedding[:,0],embedding[:,1],c=labels, alpha=0.1)

tmp = list(scatter.legend_elements())
tmp[1]=class_names 
tmp = tuple(tmp)

legend1 = ax.legend(*tmp,
                    loc="lower left", title="Classes")
ax.add_artist(legend1)

plt.xlabel('UMAP1')
plt.ylabel('UMAP2')

plt.show()

avgpool torch.Size([1892, 2048, 1, 1])


#### Zoom onto specific populations if desired

In [156]:
from oyDL import InAxes, imshow
from torch.utils.data import Subset, DataLoader
weirdDataSet = Subset(dataloader_full.dataset,indices=InAxes(ax))
weirddataloaders = DataLoader(weirdDataSet, batch_size=24,shuffle=True, num_workers=8)

fig = plt.figure()

# Get a batch of training data
inputs, classes, labels = next(iter(weirddataloaders))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs,normalize=True)

imshow(out, title=[class_names[x] for x in classes])

# Make dataset of virus infected cells

In [160]:
from oyDL import PathDataset

data_dir_hsv1 = ['/bigstore/Microscopy Core/Jen/20220104_DICCytotoxDrugsTL/predeathzoom/tnf/*',
                ]
dataset_hsv = PathDataset(data_dir_hsv1)

dataloader_hsv = torch.utils.data.DataLoader(dataset_hsv, batch_size=64,
                                              shuffle=False, num_workers=8)


In [161]:
dataloader_hsv.dataset

Dataset PathDataset
    Number of datapoints: 559
    Root location: ['/bigstore/Microscopy Core/Jen/20220104_DICCytotoxDrugsTL/predeathzoom/tnf/*']

# Show some images

In [162]:
from oyDL import imshow
fig = plt.figure()

# Get a batch of training data
inputs, classes,_ = next(iter(dataloader_hsv))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs,normalize=True)

imshow(out)

# scatter output neurons compared to labeled

In [23]:
fig = plt.figure()
ax=fig.add_subplot(111)
outcp ,labelsssss= Res.get_logits_and_labels(dataloaders['test'])

ax.scatter(outcp[:,0],outcp[:,1],c=labelsssss, alpha=0.2)
plt.xlabel(class_names[0])
plt.ylabel(class_names[1])

outcp ,_= Res.get_logits_and_labels(dataloader_hsv)

ax.scatter(outcp[:,0],outcp[:,1],c='r', alpha=0.2)
plt.xlabel(class_names[0])
plt.ylabel(class_names[1])

Text(0, 0.5, 'necro')

### You can zoom into a region and peek at some images

In [35]:
from oyDL import InAxes

weirdDataSet = Subset(dataloader_hsv.dataset,indices=InAxes(ax))
weirddataloaders = torch.utils.data.DataLoader(weirdDataSet, batch_size=16,
                                             shuffle=True, num_workers=8)

fig = plt.figure()


# Get a batch of data
inputs, classes, labels = next(iter(weirddataloaders))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs,normalize=True)

imshow(out, title=[class_names[x] for x in classes])

# scatter plot softmax

In [24]:
fig = plt.figure()
ax=fig.add_subplot(111)
outcp ,labelsssss= Res.get_softmax_and_labels(dataloaders['test'])

ax.scatter(outcp[:,0],outcp[:,1],c=labelsssss, alpha=0.2)
plt.xlabel(class_names[0])
plt.ylabel(class_names[1])

outcp ,_= Res.get_softmax_and_labels(dataloader_hsv)

ax.scatter(outcp[:,0],outcp[:,1],c='r', alpha=0.2)
plt.xlabel(class_names[0])
plt.ylabel(class_names[1])


Text(0, 0.5, 'necro')

In [163]:
plt.close('all')

## Predict labels for HSV group

In [164]:
predicts, _ = Res.get_predicts_and_labels(dataloader_hsv)
predicts = np.array(predicts)
print(class_names)
print(np.sum(np.column_stack((predicts==0, predicts==1,predicts==2)), axis=0)/len(predicts))


['apo', 'necro', 'pyro']
[0.20214669 0.48658318 0.31127013]


In [14]:
Res.visualize_model(loader=dataloader_hsv)

In [166]:
fig = plt.figure()
ax=fig.add_subplot(111)
m = nn.Softmax(dim=1)
cls=0

outcps,_= Res.get_logits_and_labels(dataloaders['test'])
h1 = ax.hist(outcps[:,cls],100)
outcps,_ = Res.get_logits_and_labels(dataloader_hsv)
h2=ax.hist(outcps[:,cls],100)


# Compare prototype UMAP to new samples

In [18]:
activations, labelsssss = Res.getActivationsByType(dataloader_full)

a = np.squeeze(np.array(activations['avgpool'].cpu()))


import umap

umapfit = umap.UMAP(n_neighbors=20,
                      min_dist=0.05,
                      metric='euclidean').fit(a)

embedding = umapfit.embedding_

fig, ax = plt.subplots()

lbls = np.unique(labels)

#y = [il for l in labels for il, L in enumerate(lbls) if l==L]
scatter = ax.scatter(embedding[:,0],embedding[:,1],c=labelsssss, alpha=0.1)

tmp = list(scatter.legend_elements())
tmp[1]=class_names #+ ['hsv']
tmp = tuple(tmp)

legend1 = ax.legend(*tmp,
                    loc="lower left", title="Classes")
ax.add_artist(legend1)

activations, labelsssss = Res.getActivationsByType(dataloader_hsv)
a = np.squeeze(np.array(activations['avgpool'].cpu()))

embedding = umapfit.transform(a)
lbls = np.unique(labels)

#y = [il for l in labels for il, L in enumerate(lbls) if l==L]
scatter = ax.scatter(embedding[:,0],embedding[:,1],c='r')

plt.show()

avgpool torch.Size([1892, 2048, 1, 1])
avgpool torch.Size([559, 2048, 1, 1])


### zoom into specific regions if desired

In [99]:
from oyDL import InAxes

#indicesThatAreWeird = [dataloader_full.dataset.indices[i] for i in InAxes()]
weirdDataSet = Subset(dataloader_hsv.dataset,indices=InAxes(ax))
weirddataloaders = torch.utils.data.DataLoader(weirdDataSet, batch_size=24,
                                             shuffle=True, num_workers=8)

fig = plt.figure()

# Get a batch of training data
inputs, classes, labels = next(iter(weirddataloaders))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs,normalize=True)

imshow(out, title=[class_names[x] for x in classes])

# 3D HERE!

In [82]:
activations, labelsssss = Res.getActivationsByType(dataloader_full)

a = np.squeeze(np.array(activations['avgpool'].cpu()))


import umap

umapfit = umap.UMAP(n_components=3, n_neighbors=20,
                      min_dist=0.1,
                      metric='euclidean').fit(a)

embedding = umapfit.embedding_

fig = plt.figure()
ax=fig.add_subplot(111,projection='3d')

lbls = np.unique(labels)

scatter = ax.scatter(embedding[:,0],embedding[:,1], embedding[:,2],c=labelsssss, alpha=0.1)

tmp = list(scatter.legend_elements())
tmp[1]=class_names# + ['hsv']
tmp = tuple(tmp)

legend1 = ax.legend(*tmp,
                    loc="lower left", title="Classes")
ax.add_artist(legend1)

activations, labelsssss = Res.getActivationsByType(dataloader_hsv)

a = np.squeeze(np.array(activations['avgpool'].cpu()))

embedding = umapfit.transform(a)
lbls = np.unique(labels)
scatter = ax.scatter(embedding[:,0],embedding[:,1], embedding[:,2],c='r')

plt.show()

avgpool torch.Size([1892, 2048, 1, 1])
avgpool torch.Size([85, 2048, 1, 1])
